Convert mol2 file from PREPI file for ATP to a PDB using Chimera.
This script takes the PDB of the ligand and combines it with the protein PDB.
1. Move protein to origin
2. Undock ligand 
3. Save only ligand 


In [44]:
# Import 
import os
import mdtraj as md
import numpy as np
from scipy.spatial.distance import cdist

In [ ]:
# Define files
#protein = "/Users/stuytschaevers/Desktop/Thesis/trwd_homolog/TrwD_intensive.pdb"
#ligand = "/Users/stuytschaevers/Desktop/Thesis/atp_mg2+/atp.pdb"
protein = "/Users/stuytschaevers/Desktop/Thesis/trwd_homolog/TrwD_intensive.pdb"
ligand = "/Users/stuytschaevers/Desktop/Thesis/atp_mg2+/amber/atp.pdb"

In [46]:
# Get trajectory of protein
protein_traj = md.load(protein)
ligand_traj = md.load(ligand)

In [49]:
# Function to move protein to origin
def move_protein_to_origin(traj):
    # Get the topology
    topology = traj.topology

    # Get protein indices
    protein_indices = [atom.index for atom in topology.atoms if atom.residue.name != "LIG"]

    # Calculate center of mass using md.compute_center_of_mass
    protein_center_of_mass = md.compute_center_of_mass(traj.atom_slice(protein_indices))[0]
    traj.xyz[0] -= protein_center_of_mass  # Translate all atoms so the protein is centered at (0,0,0)
    return traj

In [61]:
def undock_ligand(protein_traj, ligand_traj):
    # Get protein and ligand topologies
    protein_top = protein_traj.topology
    ligand_top = ligand_traj.topology

    # Get protein and ligand indices
    ligand_indices = [atom.index for atom in ligand_top.atoms if atom.residue.name == "ATP"]
    protein_indices = [atom.index for atom in protein_top.atoms if atom.residue.name != "ATP"]

    # Ensure the selections are valid
    if not ligand_indices:
        raise ValueError("Ligand selection resulted in zero atoms. Check the residue name 'ATP'.")
    if not protein_indices:
        raise ValueError("Protein selection resulted in zero atoms. Verify residue names in the PDB file.")

    # Get ligand coordinates
    ligand_coordinates = ligand_traj.xyz[0, ligand_indices]

    # Calculate ligand length (max distance between any two ligand atoms)
    ligand_length = np.max(cdist(ligand_coordinates, ligand_coordinates))

    # Calculate radii of gyration
    ligand_rg = md.compute_rg(ligand_traj.atom_slice(ligand_indices)) * 0.1  # Convert Å to nm
    protein_rg = md.compute_rg(protein_traj.atom_slice(protein_indices)) * 0.1  # Convert Å to nm
    tot_rg = ligand_rg + protein_rg
    cutoff_distance = 0.75  # Separation buffer in nm
    sphere_radius = tot_rg + 2 * cutoff_distance

    # Calculate centers of mass
    protein_center_of_mass = md.compute_center_of_mass(protein_traj.atom_slice(protein_indices))[0]
    ligand_center_of_mass = md.compute_center_of_mass(ligand_traj.atom_slice(ligand_indices))[0]

    # Calculate vector from protein to ligand and normalize
    vector = ligand_center_of_mass - protein_center_of_mass
    norm = np.linalg.norm(vector)
    if norm == 0:
        raise ValueError("Protein and ligand centers of mass coincide; cannot compute vector.")
    vector = vector / norm

    # Move ligand along the vector
    distance = sphere_radius - ligand_length - 0.25  # 0.25 nm buffer
    new_ligand_coordinates = ligand_coordinates + distance * vector

    # Update ligand trajectory
    ligand_traj.xyz[0, ligand_indices] = new_ligand_coordinates

    return ligand_traj

In [62]:
# Function to save traj
def save_traj(traj, output):
    traj.save(output)

In [63]:
def combine_protein_ligand(protein_file, ligand_file, output_file):
    # Load the protein and ligand PDB files
    protein_traj = md.load(protein_file)
    ligand_traj = md.load(ligand_file)

    # Remove hydrogen atoms from the protein
    protein_traj_noH = protein_traj.atom_slice([atom.index for atom in protein_traj.topology.atoms if atom.element.symbol != 'H'])

    # Combine the xyz coordinates of both the protein (without hydrogens) and the ligand
    combined_xyz = np.concatenate([protein_traj_noH.xyz, ligand_traj.xyz], axis=1)

    # Create a combined topology by joining the protein and ligand topologies
    combined_topology = protein_traj_noH.topology.join(ligand_traj.topology)

    # Create the combined trajectory using the new topology and combined coordinates
    combined_traj = md.Trajectory(combined_xyz, combined_topology)

    # Save the combined trajectory to the output file
    combined_traj.save(output_file)
    print(f"Combined protein and ligand saved to {output_file}")

In [64]:
# Call functions
protein_traj = move_protein_to_origin(protein_traj)
ligand_traj = undock_ligand(protein_traj, ligand_traj)
save_traj(ligand_traj, "atp_moved.pdb")
combine_protein_ligand(protein, ligand, "TrwD_atp.pdb")


Combined protein and ligand saved to TrwD_atp.pdb
